In [1]:
import os
import geopandas as gpd
import numpy as np


In [2]:
from climada_petals.hazard import WildFire
from climada.hazard import Centroids
from climada.util.constants import ONE_LAT_KM
from climada.entity import ImpactFuncSet
from climada_petals.entity.impact_funcs.wildfire import ImpfWildfire
from climada.entity import LitPop
from climada.engine import Impact
from climada.engine import ImpactCalc

In [11]:
# please make sure you run this line so that your results are reproducible
np.random.seed(42)

WF_RESOLUTION_KM = 1.0  # set resolution to 4 km
WF_RESOLUTION_ARCSEC = 30  # 120 arcsec ~ 4 km

# geographic boundaries for cities. format: (lon_min, lat_min, lon_max, lat_max)
GEO_BOUND_EU = (-31, 34, 40, 72)
GEO_BOUND_FRANCE = (-5, 41, 10, 51)

# 1. Group historical fire events using Climada

## 1.1 Load the left joined data

In [4]:
folder = '../../climada_petals/data/wildfire/output/2013/'
df = gpd.read_file(os.path.join(folder, 'merged_eu_2013_left_gdf'))
df.head()

,latitude_left,longitude_left,brightness,satellite,instrument,confidence,bright_t31,frp,daynight,index,latitude_right,longitude_right,surface,fwi,distance,date,geometry
0,35.8073,-0.2538,310.2,Aqua,MODIS,79,277.6,25.0,N,313919,35.75,-0.25,0.0,6.847656,0.057426,2013-01-01,POINT (-0.25380 35.80730)
1,47.8587,33.4466,308.0,Terra,MODIS,63,272.6,27.4,D,243494,47.75,33.50,0.0,0.214844,0.121108,2013-01-01,POINT (33.44660 47.85870)
2,49.6728,18.6611,307.4,Terra,MODIS,57,276.5,12.8,D,231915,49.75,18.75,0.0,1.417969,0.117741,2013-01-01,POINT (18.66110 49.67280)
3,37.3934,39.4902,303.1,Aqua,MODIS,55,287.0,14.4,D,302558,37.50,39.50,0.0,3.445312,0.107050,2013-01-01,POINT (39.49020 37.39340)
4,36.8729,6.9397,320.8,Terra,MODIS,75,286.9,32.9,D,306748,36.75,7.00,0.0,3.714844,0.136896,2013-01-01,POINT (6.93970 36.87290)


## 1.2 Group data by fire event

In [5]:
# Select region to group
def select_region(df, bound):
    """
    Select the region of interest from the dataframe.
    df: hazard dataframe
    bound: tuple in format (lon_min, lat_min, lon_max, lat_max)
    """
    return df[(df['longitude'] >= bound[0]) & 
              (df['latitude'] >= bound[1]) & 
              (df['longitude'] <= bound[2]) & 
              (df['latitude'] <= bound[3])]

In [6]:
# rename columns to match the required format in Climada
df.rename(columns={'date': 'acq_date', 'latitude_left': 'latitude', 'longitude_left': 'longitude'}, inplace=True)

In [7]:
df_fr = select_region(df, GEO_BOUND_FRANCE)

'''
Convert
Index([   22,    26,    27,    32,    34,    58,    59,    60,    66,    67,
       ...
       78325, 78327, 78339, 78351, 78353, 78357, 78366, 78368, 78369, 78375],
      dtype='int64', length=2814)
To
RangeIndex(start=0, stop=5224, step=1)
because Climada requires the index to be a RangeIndex
'''
df_fr.reset_index(drop=True, inplace=True)

df_fr

,latitude,longitude,brightness,satellite,instrument,confidence,bright_t31,frp,daynight,index,latitude_right,longitude_right,surface,fwi,distance,acq_date,geometry
0,43.4464,4.9050,300.8,Terra,MODIS,36,282.6,6.9,D,1306100,43.50,5.00,0.0,5.964844,0.109078,2013-01-02,POINT (4.90500 43.44640)
1,41.5108,0.3551,305.3,Terra,MODIS,61,271.6,10.6,D,1317601,41.50,0.25,0.0,10.453125,0.105653,2013-01-02,POINT (0.35510 41.51080)
2,41.4809,-1.3036,313.8,Terra,MODIS,74,275.0,22.8,D,1319035,41.50,-1.25,0.0,4.707031,0.056901,2013-01-02,POINT (-1.30360 41.48090)
3,43.4464,4.8978,304.7,Terra,MODIS,61,277.1,14.3,N,1306100,43.50,5.00,0.0,5.964844,0.115403,2013-01-02,POINT (4.89780 43.44640)
4,41.1776,1.2359,313.7,Aqua,MODIS,88,274.9,18.5,N,2357285,41.25,1.25,0.0,5.550781,0.073760,2013-01-03,POINT (1.23590 41.17760)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2809,43.4491,4.8967,304.4,Aqua,MODIS,44,283.6,14.4,D,377148980,43.50,5.00,0.0,0.398438,0.115159,2013-12-30,POINT (4.89670 43.44910)
2810,41.1857,1.2423,300.8,Aqua,MODIS,34,273.4,46.0,N,378200165,41.25,1.25,0.0,3.453125,0.064759,2013-12-31,POINT (1.24230 41.18570)
2811,43.4443,4.9006,300.8,Terra,MODIS,32,283.3,8.2,D,378187220,43.50,5.00,0.0,1.363281,0.113942,2013-12-31,POINT (4.90060 43.44430)
2812,43.3770,5.0217,324.1,Terra,MODIS,63,283.6,29.3,D,378187220,43.50,5.00,0.0,1.363281,0.124900,2013-12-31,POINT (5.02170 43.37700)


In [8]:
df_fr.index

RangeIndex(start=0, stop=2814, step=1)

In [12]:
# Set up wildfire using Climada
wf_fr = WildFire()

# Set the hazard resolution to 4 km. The provided data is at 1 km. Need a scaling factor of 1/4:
res_factor = 1./WF_RESOLUTION_KM
wf_fr.set_hist_fire_FIRMS(df_fr, centr_res_factor=0.25) 

2024-07-16 09:49:03,404 - climada_petals.hazard.wildfire - WARNING - The use of WildFire.set_hist_fire_FIRMS is deprecated.Use WildFire.from_hist_fire_FIRMS .
2024-07-16 09:49:03,415 - climada.hazard.centroids.centr - INFO - Convert centroids to GeoSeries of Point shapes.
2024-07-16 09:49:05,151 - climada_petals.hazard.wildfire - INFO - Remaining fires to identify: 1282.
2024-07-16 09:49:06,954 - climada_petals.hazard.wildfire - INFO - Remaining fires to identify: 9.
2024-07-16 09:49:07,051 - climada_petals.hazard.wildfire - INFO - Remaining fires to identify: 0.
2024-07-16 09:49:07,855 - climada_petals.hazard.wildfire - INFO - Computing intensity of 218 fires.
2024-07-16 09:49:08,189 - climada_petals.hazard.wildfire - INFO - Returning 218 fires that impacted the defined centroids.


In [13]:
wf_fr.intensity.shape

(218, 115648)

In [15]:
import pandas as pd
import numpy as np

# Sample DataFrame
data = {
    'event_id': [1, 2, 3, 1, 2],
    'latitude': [10, 20, 30, 40, 50],
    'longitude': [100, 200, 300, 400, 500],
    'brightness': [250, 260, 270, 280, 290],
    'datenum': [20200101, 20200202, 20200303, 20200404, 20200505]
}
df_firms = pd.DataFrame(data)
ev_id = 1

# Code2
temp_firms2 = df_firms.reindex(
    index=(np.argwhere(df_firms['event_id'].values == ev_id).reshape(-1,)),
    columns=['latitude', 'longitude', 'brightness', 'datenum']
)

# Modified Code1
filtered_indices = np.argwhere(df_firms['event_id'].values == ev_id).reshape(-1,)
temp_firms1 = df_firms.loc[filtered_indices, ['latitude', 'longitude', 'brightness', 'datenum']]

# Comparison
print(temp_firms1)
print(temp_firms2)
print(temp_firms1.equals(temp_firms2))  # Should print True


   latitude  longitude  brightness   datenum
0        10        100         250  20200101
3        40        400         280  20200404
   latitude  longitude  brightness   datenum
0        10        100         250  20200101
3        40        400         280  20200404
True
